In [1]:
import os
from azure.storage.blob import BlobServiceClient, ContentSettings
from dotenv import load_dotenv
load_dotenv()

True

In [5]:
# 🔧 SET THESE
connection_string = os.getenv("AZURE_CONNECTION_STRING")
container_name = os.getenv("AZURE_CONTAINER_NAME")
local_folder_path = "/Users/sharathsolomon/Desktop/SSAA Chatbot/Development/Documents" 
target_folder = "Documents"  # Azure Blob virtual folder


In [3]:
def upload_to_documents_folder(connection_string, container_name, folder_path, target_folder):
    blob_service_client = BlobServiceClient.from_connection_string(connection_string)
    container_client = blob_service_client.get_container_client(container_name)

    # Create the container if it doesn't exist
    try:
        container_client.create_container()
    except Exception:
        pass  # Ignore if it already exists

    for root, dirs, files in os.walk(folder_path):
        for filename in files:
            file_path = os.path.join(root, filename)
            relative_path = os.path.relpath(file_path, folder_path)
            blob_path = os.path.join(target_folder, relative_path).replace("\\", "/")  # Simulate folder

            # Optional: Set correct content type
            content_type = "application/octet-stream"
            if filename.endswith(".pdf"):
                content_type = "application/pdf"
            elif filename.endswith(".docx"):
                content_type = "application/vnd.openxmlformats-officedocument.wordprocessingml.document"
            elif filename.endswith(".txt"):
                content_type = "text/plain"

            try:
                with open(file_path, "rb") as data:
                    container_client.upload_blob(
                        name=blob_path,
                        data=data,
                        overwrite=True,
                        content_settings=ContentSettings(content_type=content_type),
                        timeout=300
                    )
                print(f"✅ Uploaded: {blob_path}")
            except Exception as e:
                print(f"❌ Failed to upload {blob_path}: {e}")
                continue

# ✅ CALL IT
# upload_to_documents_folder(connection_string, container_name, local_folder_path, target_folder)


In [6]:
upload_to_documents_folder(connection_string, container_name, local_folder_path, target_folder)

✅ Uploaded: Documents/0685i00000CR2gfAAD.docx
✅ Uploaded: Documents/0685i00000IxmTCAAZ.docx
✅ Uploaded: Documents/0685i00000ED4iNAAT.docx
✅ Uploaded: Documents/0685i00000KG4zZAAT.pdf
✅ Uploaded: Documents/0685i00000GMzwSAAT.pdf
✅ Uploaded: Documents/068Mo00000OA0dhIAD.pdf
✅ Uploaded: Documents/0685i00000COnTYAA1.pdf
✅ Uploaded: Documents/0685i00000CPMiuAAH.pdf
✅ Uploaded: Documents/0685i00000JTytPAAT.pdf
✅ Uploaded: Documents/068Mo00000TExCQIA1.pptx
✅ Uploaded: Documents/0685i00000CQRm5AAH.docx
✅ Uploaded: Documents/068J3000004AFZ8IAO.pdf
✅ Uploaded: Documents/0685i00000CRB13AAH.docx
✅ Uploaded: Documents/Final_Member Webinar_Storage Agreements 2023.pdf
✅ Uploaded: Documents/0685i00000CPMvQAAX.pdf
✅ Uploaded: Documents/AU-Privacy-Collection-Statement-220822-.docx
✅ Uploaded: Documents/0685i00000CQGYpAAP.docx
✅ Uploaded: Documents/0685i00000GKMkpAAH.docx
✅ Uploaded: Documents/0685i00000CR2nFAAT.docx
✅ Uploaded: Documents/0685i00000CQRgJAAX.docx
✅ Uploaded: Documents/0685i00000CQINQAA5.d

In [8]:
file_path = "/Users/sharathsolomon/Desktop/SSAA Chatbot/Development/vector_index.faiss"         # Path to the file you want to upload
blob_name = "indexer"   

# Upload file to Blob Storage
def upload_file_to_blob(file_path, blob_name,connection_string,container_name):
    blob_service_client = BlobServiceClient.from_connection_string(connection_string)
    container_client = blob_service_client.get_container_client(container_name)
    try:
        # Create a BlobClient for the target blob in the container
        blob_client = container_client.get_blob_client(blob_name)

        # Upload the file
        with open(file_path, "rb") as data:
            blob_client.upload_blob(data, overwrite=True)  # Set overwrite=True to replace existing blob if it exists

        print(f"File '{file_path}' uploaded to Blob Storage as '{blob_name}'.")

    except Exception as e:
        print(f"Error uploading file: {e}")

# Upload the file
upload_file_to_blob(file_path, blob_name,connection_string, container_name)

File '/Users/sharathsolomon/Desktop/SSAA Chatbot/Development/vector_index.faiss' uploaded to Blob Storage as 'indexer'.


In [16]:
def delete_all_blobs_in_container(connection_string, container_name):
    blob_service_client = BlobServiceClient.from_connection_string(connection_string)
    container_client = blob_service_client.get_container_client(container_name)

    blobs = container_client.list_blobs()
    for blob in blobs:
        container_client.delete_blob(blob.name)
        print(f"🗑️ Deleted: {blob.name}")

    print("✅ All blobs deleted.")

# ✅ CALL IT
connection_string = os.getenv("AZURE_CONNECTION_STRING")
container_name = os.getenv("AZURE_CONTAINER_NAME")
delete_all_blobs_in_container(connection_string, container_name)

✅ All blobs deleted.


In [4]:
model_folder = "Models"                       # Folder name where the model is stored

# Local path to save the files
local_model_folder = "./models"  # Local directory where the model files will be saved

# Initialize BlobServiceClient
blob_service_client = BlobServiceClient.from_connection_string(connection_string)

# Get the container client
container_client = blob_service_client.get_container_client(container_name)

# Function to download a file from Azure Blob Storage
def download_blob_from_storage(blob_name, download_path):
    try:
        # Get the BlobClient
        blob_client = container_client.get_blob_client(blob_name)

        # Download the blob to a local file
        with open(download_path, "wb") as download_file:
            download_file.write(blob_client.download_blob().readall())

        print(f"Downloaded '{blob_name}' to '{download_path}'.")

    except Exception as e:
        print(f"Error downloading file: {e}")

# Function to list and download all files in a folder
def download_all_files_in_folder():
    try:
        # List all blobs in the container
        blobs = container_client.list_blobs(name_starts_with=model_folder)

        for blob in blobs:
            blob_name = blob.name
            # Create the local directory structure if it doesn't exist
            local_file_path = os.path.join(local_model_folder, os.path.relpath(blob_name, model_folder))
            os.makedirs(os.path.dirname(local_file_path), exist_ok=True)

            # Download each blob
            download_blob_from_storage(blob_name, local_file_path)

    except Exception as e:
        print(f"Error downloading files: {e}")

# Create a local directory to store the model if it doesn't exist
os.makedirs(local_model_folder, exist_ok=True)

# Download all files in the 'Models' folder
download_all_files_in_folder()

blob_name = 'indexer'
download_path = './vector_index.faiss'
download_blob_from_storage(blob_name,download_path)

Downloaded 'Models/my_miniLM_model/1_Pooling/config.json' to './models/my_miniLM_model/1_Pooling/config.json'.
Downloaded 'Models/my_miniLM_model/README.md' to './models/my_miniLM_model/README.md'.
Downloaded 'Models/my_miniLM_model/config.json' to './models/my_miniLM_model/config.json'.
Downloaded 'Models/my_miniLM_model/config_sentence_transformers.json' to './models/my_miniLM_model/config_sentence_transformers.json'.
Downloaded 'Models/my_miniLM_model/model.safetensors' to './models/my_miniLM_model/model.safetensors'.
Downloaded 'Models/my_miniLM_model/modules.json' to './models/my_miniLM_model/modules.json'.
Downloaded 'Models/my_miniLM_model/sentence_bert_config.json' to './models/my_miniLM_model/sentence_bert_config.json'.
Downloaded 'Models/my_miniLM_model/special_tokens_map.json' to './models/my_miniLM_model/special_tokens_map.json'.
Downloaded 'Models/my_miniLM_model/tokenizer.json' to './models/my_miniLM_model/tokenizer.json'.
Downloaded 'Models/my_miniLM_model/tokenizer_conf